In [66]:
import polars as pl
import altair as alt       

research_ethics_system = pl.read_csv("data/4_KIOST_research_ethics_system.csv")

In [67]:
def count_y(name):
    return research_ethics_system.group_by(pl.col(name).alias("Y")).len().rename({"len":name})   

def count_list_y(columns, select):
    df = pl.DataFrame({"Y":select})
    for col in columns:
        count = research_ethics_system.group_by(pl.col(col).alias("Y")).len().rename({"len":col})
        df = df.join(count, on="Y", how="left")
    return df

def make_chart(columns):
    charts = []
    for col in columns:
        chart = alt.Chart(research_ethics_system).mark_bar().encode(
            x=f"{col}:O",
            y=f"count(ID):N",
            color=f"{col}:O"
        ).properties(
            title=f"Distribution of {col}"
        )
        charts.append(chart)
    return alt.hconcat(*charts).resolve_scale(
        y='shared'  # Synchronize the y-axis scale across all charts
    )

In [89]:
# 
# 가. KIOST의 연구윤리 관련 규정에 대한 인지 여부
# 
# Loop through each column and create a bar chart
columns = {
    "Q21_1":"연구윤리_지침", 
    "Q21_2":"동물실험_윤리",
    "Q21_3":"연구노트_관리", 
    "Q21_4":"연구과제_관리지침", 
    "Q21_5":"공직자_이해충돌_방지_제도_운영_지침"
}

system_recognition = count_list_y(columns.keys(), [1, 2]).rename(columns)
system_recognition.write_csv("figure/4/가_KIOST의_연구윤리_관련_규정에_대한_인지_여부.csv", include_bom=True)

print(system_recognition)

# chart = make_chart(columns)
# chart.show()
# chart.save("figure/4/가_KIOST의_연구윤리_관련_규정에_대한_인지_여부.svg")

shape: (2, 6)
┌─────┬───────────────┬───────────────┬───────────────┬───────────────────┬────────────────────────┐
│ Y   ┆ 연구윤리_지침 ┆ 동물실험_윤리 ┆ 연구노트_관리 ┆ 연구과제_관리지침 ┆ 공직자_이해충돌_방지_  │
│ --- ┆ ---           ┆ ---           ┆ ---           ┆ ---               ┆ 제도_운영_지침         │
│ i64 ┆ u32           ┆ u32           ┆ u32           ┆ u32               ┆ ---                    │
│     ┆               ┆               ┆               ┆                   ┆ u32                    │
╞═════╪═══════════════╪═══════════════╪═══════════════╪═══════════════════╪════════════════════════╡
│ 1   ┆ 9             ┆ 38            ┆ 8             ┆ 17                ┆ 33                     │
│ 2   ┆ 158           ┆ 129           ┆ 159           ┆ 150               ┆ 134                    │
└─────┴───────────────┴───────────────┴───────────────┴───────────────────┴────────────────────────┘


In [69]:
# 
# 나. KIOST의 연구윤리 규정의 내용 파악 정도
# 
columns = {
    "Q22_1":"연구윤리_지침", 
    "Q22_2":"동물실험_윤리",
    "Q22_3":"연구노트_관리", 
    "Q22_4":"연구과제_관리지침", 
    "Q22_5":"공직자_이해충돌_방지_제도_운영_지침"
}
counting = count_list_y(columns.keys(), [1,2,3,4,5]).drop("Y")

proportion = counting.select(
    (pl.col("Q22_1") / (pl.col("Q22_1").sum())),
    (pl.col("Q22_2") / (pl.col("Q22_2").sum())),
    (pl.col("Q22_3") / (pl.col("Q22_3").sum())),
    (pl.col("Q22_4") / (pl.col("Q22_4").sum())),
    (pl.col("Q22_5") / (pl.col("Q22_5").sum())),
).rename(columns).transpose(include_header=True).rename({"column_0":"proportion"})

counting = counting.rename(columns).transpose(include_header=True).rename({"column_0":"count"})

mean = research_ethics_system.select(
    pl.col("Q22_1").mean(),
    pl.col("Q22_2").mean(),
    pl.col("Q22_3").mean(),
    pl.col("Q22_4").mean(),
    pl.col("Q22_5").mean(),
).rename(columns).transpose(include_header=True).rename({"column_0":"mean"})

std = research_ethics_system.select(
    pl.col("Q22_1").std(),
    pl.col("Q22_2").std(),
    pl.col("Q22_3").std(),
    pl.col("Q22_4").std(),
    pl.col("Q22_5").std(),
).rename(columns).transpose(include_header=True).rename({"column_0":"std"})

all = counting.join(
    proportion, on="column", how="inner"
).join(
    mean, on="column", how="inner"
).join(
    std, on="column", how="inner"
)

print(all)

all.write_csv("figure/4/나_KIOST의_연구윤리_규정의_내용_파악_정도.csv", include_bom=True)

chart = make_chart(columns)
chart.show()
chart.save("figure/4/나_KIOST의_연구윤리_규정의_내용_파악_정도.png")

shape: (5, 13)
┌──────────────┬───────┬──────────┬──────────┬───┬─────────────┬─────────────┬──────────┬──────────┐
│ column       ┆ count ┆ column_1 ┆ column_2 ┆ … ┆ column_3_ri ┆ column_4_ri ┆ mean     ┆ std      │
│ ---          ┆ ---   ┆ ---      ┆ ---      ┆   ┆ ght         ┆ ght         ┆ ---      ┆ ---      │
│ str          ┆ u32   ┆ u32      ┆ u32      ┆   ┆ ---         ┆ ---         ┆ f64      ┆ f64      │
│              ┆       ┆          ┆          ┆   ┆ f64         ┆ f64         ┆          ┆          │
╞══════════════╪═══════╪══════════╪══════════╪═══╪═════════════╪═════════════╪══════════╪══════════╡
│ 연구윤리_지  ┆ 7     ┆ 12       ┆ 61       ┆ … ┆ 0.45509     ┆ 0.065868    ┆ 3.431138 ┆ 0.881394 │
│ 침           ┆       ┆          ┆          ┆   ┆             ┆             ┆          ┆          │
│ 동물실험_윤  ┆ 21    ┆ 46       ┆ 56       ┆ … ┆ 0.221557    ┆ 0.041916    ┆ 2.778443 ┆ 1.060809 │
│ 리           ┆       ┆          ┆          ┆   ┆             ┆             ┆          

alt.HConcatChart(...)

In [90]:
# 
# 다. 국가연구개발혁신법 인지 여부
# 

columns = ["Q23"]

law_recognition = count_y("Q23")
law_recognition.write_csv("figure/4/다_국가혁신개발법_인지_여부.csv", include_bom=True)
print(law_recognition)

chart = make_chart(columns)
chart.save("figure/4/다_국가혁신개발법_인지_여부.svg")

shape: (2, 2)
┌─────┬─────┐
│ Y   ┆ Q23 │
│ --- ┆ --- │
│ i64 ┆ u32 │
╞═════╪═════╡
│ 2   ┆ 109 │
│ 1   ┆ 58  │
└─────┴─────┘


In [71]:
# 
# 라. 국가연구개발혁신법 파악 정도
# 

# Loop through each column and create a bar chart
columns = {
    "Q24_1": "연구진실성을_보호하기_위한_노력_및_관리체계", 
    "Q24_2": "학술지_투고_학회참석_등_학문교류에_관한_윤리", 
    "Q24_3": "이해충돌_예방_및_관리",
    "Q24_4": "인간대상_연구_및_동물실험에_관한_윤리",
    "Q24_5": "연구자의_권익보호_등_건전한_연구실_문화_조성",
}

count = count_list_y(columns.keys(), [1, 2, 3])

proportion = count.select(
    (pl.col("Q24_1") / (pl.col("Q24_1").sum())),
    (pl.col("Q24_2") / (pl.col("Q24_1").sum())),
    (pl.col("Q24_3") / (pl.col("Q24_1").sum())),
    (pl.col("Q24_4") / (pl.col("Q24_1").sum())),
    (pl.col("Q24_5") / (pl.col("Q24_1").sum())),
).rename(columns).transpose(include_header=True).rename({"column_0":"proportion"})

count = count.rename(columns).transpose(include_header=True).rename({"column_0":"count"})

all = count.join(
    proportion, on="column", how="inner"
)

print(all)
all.write_csv("figure/4/라_국가혁신개발법_이해_여부.csv", include_bom=True)

charts = make_chart(columns)
charts.save("figure/4/라_국가혁신개발법_이해_여부.png")

shape: (5, 7)
┌─────────────────────┬───────┬──────────┬──────────┬────────────┬────────────────┬────────────────┐
│ column              ┆ count ┆ column_1 ┆ column_2 ┆ proportion ┆ column_1_right ┆ column_2_right │
│ ---                 ┆ ---   ┆ ---      ┆ ---      ┆ ---        ┆ ---            ┆ ---            │
│ str                 ┆ i64   ┆ i64      ┆ i64      ┆ f64        ┆ f64            ┆ f64            │
╞═════════════════════╪═══════╪══════════╪══════════╪════════════╪════════════════╪════════════════╡
│ 연구진실성을_보호하 ┆ 98    ┆ 2        ┆ 67       ┆ 0.586826   ┆ 0.011976       ┆ 0.401198       │
│ 기_위한_노력_및_관  ┆       ┆          ┆          ┆            ┆                ┆                │
│ 리체계              ┆       ┆          ┆          ┆            ┆                ┆                │
│ 학술지_투고_학회참  ┆ 106   ┆ 4        ┆ 57       ┆ 0.634731   ┆ 0.023952       ┆ 0.341317       │
│ 석_등_학문교류에_관 ┆       ┆          ┆          ┆            ┆                ┆                │
│ 한_윤리     

In [72]:
# 
# 마. 연구부정행위 위반 인지
# 

columns = {
    "Q25_1": "위조", 
    "Q25_2": "변조", 
    "Q25_3": "표절", 
    "Q25_4": "부당한 저자표시", 
    "Q25_5": "중복게재", 
    "Q25_6": "조사방해 행위, 제보자 위협 행위 등의 기타", 
    "Q25_7": "동물실험윤리", 
    "Q25_8": "연구노트 관리", 
    "Q25_9": "연구 성과 관리 및 연구 보안", 
    "Q25_10": "연구비 집행과 관리"
}

count = count_list_y(columns.keys(), [1, 2])

proportion = count.select(
    pl.col("Q25_1") / pl.col("Q25_1").sum(),
    pl.col("Q25_2") / pl.col("Q25_2").sum(),
    pl.col("Q25_3") / pl.col("Q25_3").sum(),
    pl.col("Q25_4") / pl.col("Q25_4").sum(),
    pl.col("Q25_5") / pl.col("Q25_5").sum(),
    pl.col("Q25_6") / pl.col("Q25_6").sum(),
    pl.col("Q25_7") / pl.col("Q25_7").sum(),
    pl.col("Q25_8") / pl.col("Q25_8").sum(),
    pl.col("Q25_9") / pl.col("Q25_9").sum(),
    pl.col("Q25_10") / pl.col("Q25_10").sum(),
).rename(columns).transpose(include_header=True).rename({"column_0":"proportion"})

count = count.rename(columns).transpose(include_header=True).rename({"column_0":"count"})

all = count.join(
    proportion, on="column", how="inner"
)
print(all)
all.write_csv("figure/4/마_연구부정행위_위반_인지.csv", include_bom=True)

chart = make_chart(columns)
chart.save("figure/4/마_연구부정행위_위반_인지.svg")

shape: (10, 5)
┌─────────────────────────────────┬───────┬──────────┬────────────┬────────────────┐
│ column                          ┆ count ┆ column_1 ┆ proportion ┆ column_1_right │
│ ---                             ┆ ---   ┆ ---      ┆ ---        ┆ ---            │
│ str                             ┆ i64   ┆ i64      ┆ f64        ┆ f64            │
╞═════════════════════════════════╪═══════╪══════════╪════════════╪════════════════╡
│ 위조                            ┆ 10    ┆ 157      ┆ 0.05988    ┆ 0.94012        │
│ 변조                            ┆ 8     ┆ 159      ┆ 0.047904   ┆ 0.952096       │
│ 표절                            ┆ 9     ┆ 158      ┆ 0.053892   ┆ 0.946108       │
│ 부당한 저자표시                 ┆ 27    ┆ 140      ┆ 0.161677   ┆ 0.838323       │
│ 중복게재                        ┆ 14    ┆ 153      ┆ 0.083832   ┆ 0.916168       │
│ 조사방해 행위, 제보자 위협 행위 ┆ 5     ┆ 162      ┆ 0.02994    ┆ 0.97006        │
│ 등의 기타                       ┆       ┆          ┆            ┆                │

In [92]:
# 
# 바. 연구부정행위 위반 내용 보고 방법
# 

# 26. 귀하는 연구부정행위를 인지하게 되었을 때 KIOST 또는 외부의 관련 기관에 제보하는 방법을 알고 있습니까?
# 27. 귀하는 연구부정행위에 대한 제보가 접수된 후 어떻게 검증되는지 그 절차에 대해 알고 있습니까?
columns = {
    "Q26": "연구부정행위 제보 방법",
    "Q27": "연구부정행위 제보의 검증 절차"
}
count = count_list_y(columns.keys(), [1, 2])

print(count)

proportion = count.select(
    pl.col("Q26") / pl.col("Q26").sum(),
    pl.col("Q27") / pl.col("Q27").sum(),
).rename(columns).transpose(include_header=True).rename({"column_0":"proportion"})

count = count.rename(columns).transpose(include_header=True).rename({"column_0":"count"})
all = count.join(
    proportion, on="column", how="inner"
)
print(all)

all.write_csv("figure/4/바_연구부정행위_위반_내용_보고_방법.csv", include_bom=True)
charts = make_chart(columns)
charts.save("figure/4/바_연구부정행위_위반_내용_보고_방법.png")

shape: (2, 3)
┌─────┬─────┬─────┐
│ Y   ┆ Q26 ┆ Q27 │
│ --- ┆ --- ┆ --- │
│ i64 ┆ u32 ┆ u32 │
╞═════╪═════╪═════╡
│ 1   ┆ 87  ┆ 50  │
│ 2   ┆ 80  ┆ 117 │
└─────┴─────┴─────┘
shape: (2, 5)
┌───────────────────────────────┬───────┬──────────┬────────────┬────────────────┐
│ column                        ┆ count ┆ column_1 ┆ proportion ┆ column_1_right │
│ ---                           ┆ ---   ┆ ---      ┆ ---        ┆ ---            │
│ str                           ┆ i64   ┆ i64      ┆ f64        ┆ f64            │
╞═══════════════════════════════╪═══════╪══════════╪════════════╪════════════════╡
│ 연구부정행위 제보 방법        ┆ 87    ┆ 80       ┆ 0.520958   ┆ 0.479042       │
│ 연구부정행위 제보의 검증 절차 ┆ 50    ┆ 117      ┆ 0.299401   ┆ 0.700599       │
└───────────────────────────────┴───────┴──────────┴────────────┴────────────────┘


In [94]:
# 
# 사. KIOST의 연구부정행위에 대한 제보 접수 및 처리 규정 준수
# 
columns = {
    "Q28" : "연구부정행위 제보 접수 및 처리 규정 준수"
}
count = count_list_y(columns, [1, 2, 3, 4, 5])

print(count)

proportion = count.select(
    pl.col("Q28") / pl.col("Q28").sum(),
).rename(columns).transpose(include_header=True).rename({"column_0":"proportion"})

count = count.rename(columns).transpose(include_header=True).rename({"column_0":"count"})
all = count.join(
    proportion, on="column", how="inner"
)

print(all)

all.write_csv("figure/4/사_KIOST의_연구부정행위에_대한_제보_접수_및_처리_규정_준수.csv", include_bom=True)

charts = make_chart(columns)
charts.save("figure/4/사_KIOST의_연구부정행위에_대한_제보_접수_및_처리_규정_준수.svg")

shape: (5, 2)
┌─────┬──────┐
│ Y   ┆ Q28  │
│ --- ┆ ---  │
│ i64 ┆ u32  │
╞═════╪══════╡
│ 1   ┆ null │
│ 2   ┆ 15   │
│ 3   ┆ 72   │
│ 4   ┆ 54   │
│ 5   ┆ 26   │
└─────┴──────┘
shape: (1, 11)
┌────────────┬───────┬──────────┬──────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ column     ┆ count ┆ column_1 ┆ column_2 ┆ … ┆ column_1_r ┆ column_2_r ┆ column_3_r ┆ column_4_r │
│ ---        ┆ ---   ┆ ---      ┆ ---      ┆   ┆ ight       ┆ ight       ┆ ight       ┆ ight       │
│ str        ┆ i64   ┆ i64      ┆ i64      ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│            ┆       ┆          ┆          ┆   ┆ f64        ┆ f64        ┆ f64        ┆ f64        │
╞════════════╪═══════╪══════════╪══════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 연구부정행 ┆ null  ┆ 15       ┆ 72       ┆ … ┆ 0.08982    ┆ 0.431138   ┆ 0.323353   ┆ 0.155689   │
│ 위 제보    ┆       ┆          ┆          ┆   ┆            ┆            ┆            ┆            │
│ 접수 및

In [75]:
# 
# 아. KIOST의 연구부정행위 검증에 대한 객관성 및 공정성
# 
columns = {
    "Q29": "연구부정행위 검증에 대한 객관성 및 공정성"
}
count = count_list_y(columns, [1, 2, 3, 4, 5])

proportion = count.select(
    pl.col("Q29") / pl.col("Q29").sum(),
).rename(columns).transpose(include_header=True).rename({"column_0":"proportion"})

count = count.rename(columns).transpose(include_header=True).rename({"column_0":"count"})
all = count.join(
    proportion, on="column", how="inner"
)
all.write_csv("figure/4/아_KIOST의_연구부정행위_검증에_대한_객관성_및_공정성.csv", include_bom=True)
charts = make_chart(columns)
charts.save("figure/4/아_KIOST의_연구부정행위_검증에_대한_객관성_및_공정성.svg")

In [76]:
def basic_analysis(name):
    count = research_ethics_system.group_by(name).len().sort(name)
    proportion = count.with_columns(
        pct = pl.col("len") / pl.col("len").sum() * 100
    )
    mean = proportion.with_columns(
        mean = (pl.col(name) * pl.col("len")).sum() / pl.col("len").sum()
    )
    std = mean.with_columns(
        std = (((pl.col(name) - pl.col("mean")) ** 2 * pl.col("len")).sum() / pl.col("len").sum()).sqrt(),
    )

    return std

In [77]:
from attributes import *
from scipy import stats

#
# Chi-square test
#
personal_info = pl.read_csv("data/6_personal_info.csv")
recognition_stats = research_ethics_system.join(personal_info, on="ID")

def chi2_test(target_name):
    chi_analysis = {}
    for key, value in MAP_PERSON.items():
        target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)
        t, p, dof, _ = stats.chi2_contingency(target.to_numpy())
        chi_analysis[key] = (float(t), float(p))
    return chi_analysis

In [78]:
# 
# 자. KIOST 내 연구자의 연구윤리 준수 상황에 대한 평가: (최근 3년 간 KIOST 연구자의 연구윤리 준수 상황이 어떠하다고 생각하십니까?)
# 
# 1) 연구진실성
title = "Q31_1"
print(chi2_test(title))
basic_analysis(title) 

{'성별': (9.592872684237355, 0.022363602346016385), '연령대': (18.651282149555605, 0.0972987726366444), '직위': (31.345117303751223, 0.00789616027251365), '최종학위': (10.039642639377995, 0.12299247851397707), '연구경력': (12.240856718081318, 0.20006904038716064), '연구분야': (21.328517805215437, 0.2631640451306318)}


C:\Users\hyeon\AppData\Local\Temp\ipykernel_20680\3864724766.py:13: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


Q31_1,len,pct,mean,std
i64,u32,f64,f64,f64
2,3,1.796407,3.856287,0.719909
3,48,28.742515,3.856287,0.719909
4,86,51.497006,3.856287,0.719909
5,30,17.964072,3.856287,0.719909


In [79]:
# 
# 자. KIOST 내 연구자의 연구윤리 준수 상황에 대한 평가: (최근 3년 간 KIOST 연구자의 연구윤리 준수 상황이 어떠하다고 생각하십니까?)
# 
# 2) 동물실험윤리
title = "Q31_2"
print(chi2_test(title))
basic_analysis(title) 

{'성별': (5.214634851995489, 0.15673850837772643), '연령대': (18.150133395453544, 0.11120741621069025), '직위': (26.08478915123719, 0.03714085359429702), '최종학위': (9.922501644419341, 0.12795381112365045), '연구경력': (9.392329338422932, 0.40187463750403674), '연구분야': (21.221339686287553, 0.26839363946962597)}


C:\Users\hyeon\AppData\Local\Temp\ipykernel_20680\3864724766.py:13: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


Q31_2,len,pct,mean,std
i64,u32,f64,f64,f64
2,4,2.39521,3.826347,0.733967
3,50,29.94012,3.826347,0.733967
4,84,50.299401,3.826347,0.733967
5,29,17.365269,3.826347,0.733967


In [80]:
# 
# 자. KIOST 내 연구자의 연구윤리 준수 상황에 대한 평가: (최근 3년 간 KIOST 연구자의 연구윤리 준수 상황이 어떠하다고 생각하십니까?)
# 
# 3) 연구노트관리
title = "Q31_3"
print(chi2_test(title))
basic_analysis(title) 

{'성별': (8.92360612849981, 0.06303758788186091), '연령대': (12.278245775393241, 0.7246140304049122), '직위': (35.19079759050956, 0.019111327017450764), '최종학위': (25.01521694433737, 0.0015453546166226161), '연구경력': (11.645029156508189, 0.47459218518348534), '연구분야': (28.921820872276943, 0.223087023098122)}


C:\Users\hyeon\AppData\Local\Temp\ipykernel_20680\3864724766.py:13: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


Q31_3,len,pct,mean,std
i64,u32,f64,f64,f64
1,1,0.598802,3.886228,0.761492
2,3,1.796407,3.886228,0.761492
3,44,26.347305,3.886228,0.761492
4,85,50.898204,3.886228,0.761492
5,34,20.359281,3.886228,0.761492


In [81]:
# 
# 자. KIOST 내 연구자의 연구윤리 준수 상황에 대한 평가: (최근 3년 간 KIOST 연구자의 연구윤리 준수 상황이 어떠하다고 생각하십니까?)
# 
# 4) 연구비 집행 및 관리
title = "Q31_4"
print(chi2_test(title))
basic_analysis(title) 

{'성별': (7.801429511131103, 0.09912895580312492), '연령대': (20.810589258214055, 0.1859067711467114), '직위': (37.501736490644305, 0.010181514195794201), '최종학위': (28.855746864627758, 0.0003361199655696349), '연구경력': (20.356531787732564, 0.060635074610903686), '연구분야': (29.860257594553644, 0.18942660902542283)}


C:\Users\hyeon\AppData\Local\Temp\ipykernel_20680\3864724766.py:13: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


Q31_4,len,pct,mean,std
i64,u32,f64,f64,f64
1,1,0.598802,3.91018,0.748862
2,3,1.796407,3.91018,0.748862
3,40,23.952096,3.91018,0.748862
4,89,53.293413,3.91018,0.748862
5,34,20.359281,3.91018,0.748862


In [82]:
# 
# 자-2) 31 종속 문항, 연구윤리 위반 발생 이유 
# (31-1~4 항목에 대해 전반적으도 ‘나빠졌다’ 또는 ‘매우 나빠졌다’고 응답한 경우) 귀하는 KIOST 연구자의 연구윤리 위반이 왜 발생한다고 생각하십니까?* 전반적으로 그대로다, 좋아졌다, 매우좋아졌다 답한경우 무응답 가능
# 
# 1.연구윤리 의식의 부족
# 2.연구윤리 교육의 부족
# 3.잘못된 연구의 관행 문제
# 4.연구윤리 관련 제도의 미흡
# 5.연구자의 연구윤리에 대한 무관심과 일탈 문제
# 6.많은 연구업적 산출에 대한 스트레스와 지나친 경쟁
# 7.연구실 또는 기관 내 연구자간 갈등과 소통 부재
# 8.기타 
# 
title = "Q32"
print(chi2_test(title))
basic_analysis(title)

{'성별': (10.872087760724124, 0.14429662857790673), '연령대': (26.94270137320409, 0.5213689746833431), '직위': (49.49246551724138, 0.05310008104058568), '최종학위': (17.58492841016426, 0.22634387041144347), '연구경력': (23.60159855808352, 0.3127597064490119), '연구분야': (42.619957465678, 0.44431217236962534)}


C:\Users\hyeon\AppData\Local\Temp\ipykernel_20680\3864724766.py:13: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


Q32,len,pct,mean,std
i64,u32,f64,f64,f64
null,140,83.832335,0.754491,1.704587
1,1,0.598802,0.754491,1.704587
2,1,0.598802,0.754491,1.704587
3,7,4.191617,0.754491,1.704587
4,2,1.197605,0.754491,1.704587
5,5,2.994012,0.754491,1.704587
6,8,4.790419,0.754491,1.704587
7,3,1.796407,0.754491,1.704587


In [83]:
# 
# 자-3) 31 종속 문항, 연구윤리 위반 발생 이유 
# (31-1~4 항목에 대해 전반적으도 ‘나빠졌다’ 또는 ‘매우 나빠졌다’고 응답한 경우) 귀하는 KIOST 연구자의 연구윤리 위반을 예방하기 위해 무엇이 필요하다고 보십니까?* 전반적으로 그대로다, 좋아졌다, 매우좋아졌다 답한경우 무응답 가능
# 
# 1.연구윤리 교육 강화
# 2.연구윤리 확립을 위한 제도 개선
# 3.연구자의 연구윤리 의식 개선
# 4.건전한 연구윤리 문화(분위기) 정립
# 5.연구원간 상호존중과 민주적인 의사소통
# 6.양적 위주의 업적평가 시스템 개선
# 7.기타 
# 
title = "Q33"
print(chi2_test(title))
basic_analysis(title)

{'성별': (6.357711209489866, 0.4986562015574849), '연령대': (30.670969481189896, 0.33188322421933186), '직위': (31.988674309777757, 0.6142540669809582), '최종학위': (6.349619378864662, 0.9568953465347328), '연구경력': (21.51766051155149, 0.42774577089995486), '연구분야': (48.04735490972277, 0.2411646461168693)}


C:\Users\hyeon\AppData\Local\Temp\ipykernel_20680\3864724766.py:13: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


Q33,len,pct,mean,std
i64,u32,f64,f64,f64
null,111,66.467066,1.341317,1.804587
1,4,2.39521,1.341317,1.804587
2,5,2.994012,1.341317,1.804587
3,14,8.383234,1.341317,1.804587
4,14,8.383234,1.341317,1.804587
5,6,3.592814,1.341317,1.804587
6,9,5.389222,1.341317,1.804587
7,4,2.39521,1.341317,1.804587


In [87]:
# 
# 귀하는 부실학술활동(부실학술지에 논문 투고, 부실한 학술대회에 참석하기)에 참여하는 것이 연구부정행위라고 생각하십니까?
# 

title="Q34"
print(chi2_test(title))
basic_analysis(title)

{'성별': (6.688363231021785, 0.15330218725555064), '연령대': (28.28460309631121, 0.029231058848692208), '직위': (53.232222927136725, 7.512122129885235e-05), '최종학위': (35.77240513388311, 1.9327197755419556e-05), '연구경력': (33.10854227593121, 0.0009309704687208946), '연구분야': (23.59303110761058, 0.48505533222272934)}


C:\Users\hyeon\AppData\Local\Temp\ipykernel_20680\3864724766.py:13: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


Q34,len,pct,mean,std
i64,u32,f64,f64,f64
1,3,1.796407,3.826347,0.990869
2,16,9.580838,3.826347,0.990869
3,33,19.760479,3.826347,0.990869
4,70,41.916168,3.826347,0.990869
5,45,26.946108,3.826347,0.990869


In [88]:
# 
# 귀하는 ChatGPT 등의 생성형 AI를 활용하여 연구를 수행한 적이 있습니까?
# 

title="Q35"
print(chi2_test(title))
basic_analysis(title)

{'성별': (7.478658859413192, 0.11265403717463955), '연령대': (39.66210294210465, 0.000872059915148659), '직위': (49.228943639110284, 0.000285289010138137), '최종학위': (7.445763800176664, 0.48938774429862797), '연구경력': (40.44151283088866, 6.072750116956202e-05), '연구분야': (48.68387371894782, 0.0020750797080728166)}


C:\Users\hyeon\AppData\Local\Temp\ipykernel_20680\3864724766.py:13: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


Q35,len,pct,mean,std
i64,u32,f64,f64,f64
1,45,26.946108,2.646707,1.327182
2,38,22.754491,2.646707,1.327182
3,29,17.365269,2.646707,1.327182
4,41,24.550898,2.646707,1.327182
5,14,8.383234,2.646707,1.327182


In [86]:
from attributes import *
from scipy import stats

#
# Chi-square test
#
personal_info = pl.read_csv("data/6_personal_info.csv")
recognition_stats = research_ethics_system.join(personal_info, on="ID")

def run_chi2_test(target_name):
    chi_analysis = {}
    for key, value in MAP_PERSON.items():
        target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)
        t, p, dof, _ = stats.chi2_contingency(target.to_numpy())
        chi_analysis[key] = {f"{key}_t":t, f"{key}_p":p}
    return chi_analysis

def run_chi(column, name):
    dataframe = pl.DataFrame()
    for c in column:
        result = pl.from_dict(run_chi2_test(c)) \
                    .unnest("sex") \
                    .unnest("age") \
                    .unnest("role") \
                    .unnest("degree") \
                    .unnest("experience") \
                    .unnest("field")
        dataframe = dataframe.vstack(result)
    dataframe.write_csv(f"figure/4/{name}_chi.csv", include_bom=True)


q21 = ["Q21_1", "Q21_2", "Q21_3", "Q21_4", "Q21_5"]
q22 = ["Q22_1", "Q22_2", "Q22_3", "Q22_4", "Q22_5"]
q23 = ["Q23"]
q24 = ["Q24_1", "Q24_2", "Q24_3", "Q24_4", "Q24_5"]
q25 = ["Q25_1", "Q25_2", "Q25_3", "Q25_4", "Q25_5", "Q25_6", "Q25_7", "Q25_8", "Q25_9", "Q25_10"]
q26 = ["Q26"]
q27 = ["Q27"]
q28 = ["Q28"]
q29 = ["Q29"]

run_chi(q21, "가_연구윤리_규정_인지_검정")
run_chi(q22, "나_연구윤리_규정_파악")
run_chi(q23, "다_국가혁신개발법_인지")
run_chi(q24, "라_국가혁신개발법_이해")
run_chi(q25, "마_연구부정행위_인지")
run_chi(q26, "바_연구부정행위_제보")
run_chi(q27, "바_연구부정행위_검증")
run_chi(q28, "사_연구부정행위_제보")
run_chi(q29, "아_연구부정행위_검증")

C:\Users\hyeon\AppData\Local\Temp\ipykernel_20680\4186764624.py:13: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


ColumnNotFoundError: "sex" not found